In [2]:
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
import os

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

print(torch.cuda.is_available())
device = "cuda:0"
name = "fashion_mnist"

True


In [3]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_fashion_mnist.pth')
estado_fase_1 = torch.load('Fase_1_fashion_mnist.pth')


In [4]:
print(estado_fase_1["linear_relu_stack.4.weight"])
print(estado_fase_1["linear_relu_stack.4.bias"])

tensor([[ 0.0295, -0.0629, -0.0960,  ...,  0.0392, -0.0191, -0.0023],
        [-0.0304, -0.0987,  0.1255,  ...,  0.0932,  0.0156,  0.0661],
        [ 0.0454, -0.0399, -0.0400,  ..., -0.0902, -0.0465, -0.0579],
        ...,
        [-0.0757,  0.0191,  0.0644,  ...,  0.0255,  0.0509, -0.0672],
        [ 0.0554,  0.0415, -0.0724,  ..., -0.0260, -0.0355, -0.0522],
        [ 0.0601,  0.0156,  0.0552,  ..., -0.0438, -0.0592,  0.0380]],
       device='cuda:0')
tensor([ 0.0672,  0.0397, -0.0077,  0.0239, -0.1433,  0.4709, -0.0521,  0.0403,
        -0.2079, -0.2249], device='cuda:0')


## Change f_0(5-9) to f_1(5-9) MODEL 1

In [5]:

estado_fase_1["linear_relu_stack.4.weight"][5:10, :] = estado_fase_0["linear_relu_stack.4.weight"][5:10, :]
estado_fase_1["linear_relu_stack.4.bias"][5:10] = estado_fase_0["linear_relu_stack.4.bias"][5:10]

txt = f"logs/preditions_combination_classifier_f1_0_4_f0_5_9_F1_{name}.txt"

In [6]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_1)

<All keys matched successfully>

## Change f_0(0-4) to f_1(0-4) MODEL 1

In [4]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_fashion_mnist.pth')
estado_fase_1 = torch.load('Fase_1_fashion_mnist.pth')

In [5]:

# Transferir el clasificador de Fase_0 a Fase_1
a, b = 0, 5
estado_fase_1["linear_relu_stack.4.weight"][a:b, :] = estado_fase_0["linear_relu_stack.4.weight"][a:b, :]
estado_fase_1["linear_relu_stack.4.bias"][a:b] = estado_fase_0["linear_relu_stack.4.bias"][a:b]

txt = f"logs/preditions_combination_classifier_f0_0_4_f1_5_9_F1_{name}.txt"

In [6]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_1)

<All keys matched successfully>

## Change f_1(0-4) to f_0(0,4) MODEL 0

In [10]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_fashion_mnist.pth')
estado_fase_1 = torch.load('Fase_1_fashion_mnist.pth')

In [11]:

a, b = 0, 5
estado_fase_0["linear_relu_stack.4.weight"][a:b, :] = estado_fase_1["linear_relu_stack.4.weight"][a:b, :]
estado_fase_0["linear_relu_stack.4.bias"][a:b] = estado_fase_1["linear_relu_stack.4.bias"][a:b]

txt = f"logs/preditions_combination_classifier_f1_0_4_f0_5_9_F0_{name}.txt"

In [12]:
model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_0)

<All keys matched successfully>

## Change f_1(5-9) to f_1(5-9) MODEL 0

In [16]:
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0_fashion_mnist.pth')
estado_fase_1 = torch.load('Fase_1_fashion_mnist.pth')

In [17]:

# Transferir el clasificador de Fase_0 a Fase_1
a, b = 5, 10
estado_fase_0["linear_relu_stack.4.weight"][a:b, :] = estado_fase_1["linear_relu_stack.4.weight"][a:b, :]
estado_fase_0["linear_relu_stack.4.bias"][a:b] = estado_fase_1["linear_relu_stack.4.bias"][a:b]

txt = f"logs/preditions_combination_classifier_f0_0_4_f1_5_9_F0_{name}.txt"

In [18]:

model_classifier = NeuralNetwork().to(device)
model_classifier.load_state_dict(estado_fase_0)

<All keys matched successfully>

### Evaluación

In [13]:
def test_loop(dataloader, model, loss_fn, txt):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tarjet_prediction = []  # Lista para almacenar las etiquetas reales y predicciones

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Guardar la etiqueta real y la predicción en la lista de tuplas
            tarjet_prediction.extend(list(zip(y.cpu().numpy(), pred.argmax(1).cpu().numpy())))

    test_loss /= num_batches
    correct /= size
    log_accuracy_loss.append((100*correct, test_loss))
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

    with open(f'{txt}', 'w') as archivo:
            # Escribe el valor de la variable en el archivo
            archivo.write(str(tarjet_prediction))
    print(f'El valor prediciones se ha guardado en el archivo.txt')
    

In [19]:
eval_data = torchvision.datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
eval_dataloader = DataLoader(eval_data, batch_size=10000, shuffle=True)
print("Se cargaron los datos correctamente")

Se cargaron los datos correctamente


In [21]:
learning_rate = 1e-3
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_classifier.parameters(), lr=learning_rate)
epochs = 1
log_accuracy_loss = []
test_loop(eval_dataloader, model_classifier, loss_fn, txt)

Test Error: 
 Accuracy: 41.1%, Avg loss: 2.610565 

El valor prediciones se ha guardado en el archivo.txt
